# Data Cleaning

<div style="background-color:#f4f4f4; padding: 20px; border-radius: 10px;">

<h3>Understanding Missing Values, Duplicates, and Necessary Drops</h3>

<ul>
  <li><strong>Dataset Overview</strong>: 12,495 entries and 169 attributes.</li>
  <li><strong>Missing Values</strong>: 217 missing values in <code>PRICE</code> column.</li>
  <li><strong>Empty Columns</strong>: <code>GROUP_META</code> contains only null values.</li>
  <li><strong>Data Completeness</strong>: Except <code>location</code>, each column has at least 217 missing values. Additionally, each row has at least 36 missing values.</li>
  <li><strong>Duplicates</strong>: Identified 7 sets of duplicate columns.</li>
  <li><strong>Cleaning Actions</strong>:
    <ul>
      <li>Removed 127 unnecessary columns.</li>
      <li>Dropped 2 rows with missing <code>AREA</code> and <code>PRICE</code> within a specific range.</li>
      <li>Eliminated 73 rows with 'price on request' in <code>PRICE</code> column.</li>
    </ul>
  </li>
</ul>

<h3>Correcting Columns, Introducing New Features, and Necessary Drops</h3>

<ul>
  <li><strong>Data Refinement</strong>: Reduced to 12,409 entries and 42 attributes.</li>
  <li><strong>Correcting Entries</strong>:
    <ul>
      <li>Fixed <code>AREA</code> by removing 'sq.ft.'.</li>
      <li>Adjusted <code>PRICE</code> by replacing 'Cr' or 'L' with correct values.</li>
    </ul>
  </li>
  <li><strong>New Feature</strong>: Introduced <code>AREA_TYPE</code> indicating the type of area.</li>
  <li><strong>Cleanup</strong>: Removed 20 unnecessary columns.</li>
</ul>

<h3>Dataset Reduction and Focus on Residential Apartments</h3>

<ul>
  <li><strong>Refocused Dataset</strong>: Down to 12,409 entries and 22 attributes.</li>
  <li><strong>Property Types</strong>: Identified 8 types but focused on 'Residential Apartment'.</li>
  <li><strong>Extraction</strong>: Segregated other types for future analysis.</li>
  <li><strong>Focus</strong>: Currently analyzing only 'Residential Apartment' category, totaling 9,291 entries.</li>
</ul>

<h3>Data Cleaning and Preparation for Residential Apartment Dataset Analysis</h3>

<ul>
  <li><strong>Data Preparation</strong>: 9,291 entries and 22 attributes.</li>
  <li><strong>Handling Missing Data</strong>:
    <ul>
      <li>Removed 5 rows with null values in specific columns.</li>
      <li>Imputed missing values in several columns.</li>
    </ul>
  </li>
  <li><strong>Feature Engineering</strong>: Introduced <code>FURNISH_LABEL</code>.</li>
  <li><strong>Column Adjustment</strong>:
    <ul>
      <li>Corrected error in <code>CITY</code> column.</li>
      <li>Mapped values in <code>FLOOR_NUM</code>.</li>
      <li>Converted <code>AREA</code> values to float.</li>
    </ul>
  </li>
  <li><strong>Cleanup</strong>: Removed unnecessary column.</li>
  <li><strong>Saving</strong>: Saved as <code>Residential_Apartment.csv</code> for EDA.</li>
</ul>

</div>

In [1]:
import re
import ast
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
df = pd.read_csv('data.csv', low_memory=False, index_col=0)
dff = df.copy()

## Understanding Missing Values, Duplicates, and Necessary Drops

- Dataset contains 12,495 entries and 169 different pieces of information. 
- In the `PRICE` column, there are 217 missing values. 
- The `GROUP_META` column has no data; it's entirely composed of null values. 
- However, the `location` column is fully populated; there are no missing values there. 
- In fact, each column, except for `location`, has at least 217 missing values. 
- Additionally, each row has at least 36 missing values across all columns. 
- I've identified 7 sets of duplicate columns. 
- To clean up the dataset, we're going to remove 127 unnecessary columns. 
- Furthermore, I've eliminated 2 rows where both the `AREA` and `PRICE` fall within a certain range. 
- Finally, I've drop 73 rows where the `PRICE` value is listed as 'price on request'.

In [4]:
df.shape

(12498, 169)

In [5]:
df.duplicated().sum()

0

In [6]:
df.isnull().all(axis=1).sum()

0

In [7]:
missing_values_col_df = ((1 - df.count() / len(df))*100).sort_values(ascending=False).to_frame(name='Missing Values Percentage')

In [8]:
missing_values_col_df.sort_values('Missing Values Percentage').head(1)

,Missing Values Percentage
location,0.0


In [9]:
missing_values_col_df.head(1)

,Missing Values Percentage
GROUP_META,100.0


In [10]:
missing_values_col_df.loc['PRICE']

Missing Values Percentage    2.160346
Name: PRICE, dtype: float64

In [11]:
missing_values_row_df = (df.isnull().sum(axis=1).sort_values(ascending=False)).to_frame(name='Number of missing cols')

In [12]:
missing_values_row_df.value_counts().sort_index()[:1]

Number of missing cols
36                        1
Name: count, dtype: int64

In [13]:
column_groups = {}

for column in df.columns:
    column_values = tuple(df[column])
    if column_values in column_groups:
        column_groups[column_values].append(column)
    else:
        column_groups[column_values] = [column]

duplicate_column_groups = [group for group in column_groups.values() if len(group) > 1]

if duplicate_column_groups:
    for group in duplicate_column_groups:
        print(group)

['PROP_NAME', 'SOCIETY_NAME', 'BUILDING_NAME']
['IS_NEW_LAUNCH', 'SHORTLISTED', 'REPORTED', 'RTOV_BOOKED']
['IS_POSTER_RERA_REGISTERED', 'IS_DEALER_RERA_REGISTERED']
['EXPIRY_DATE', 'EXPIRY_DATE__U']
['OFFER_TAGS', 'PRELEASED_TAGS', 'PRELEASE_DATA']
['caraousalImages', 'configCardTuples']
['BATHROOM_ATTACHED', 'BALCONY_ATTACHED']


In [14]:
df.drop(
    columns = [
        'AREA_UNIT__U', 'BUILDING_NAME', 'PROPERTY_VIDEOS', 'page', 'REGISTER_DATE_FORMATTED',
        'price', 'PRELEASED_TAGS', 'PROJ_ID', 'reraDisplayTag', 'RES_COM', 'BUILDING_ID', 'eoi',
        'GROUP_COUNT', 'entityType', 'AVAILABILITY', 'metadata', 'PD_URL', 'description',
        'TRANSACT_TYPE', 'CONTACT_NAME', 'rating', 'FURNISH', 'APPROVED_AUTHORITIES', 'rescom',
        'keyHighlights', 'mediaCount', 'EXPIRY_DATE__U', 'PRELEASE_DATA', 'INFINITY_VIDEO_SCREENING',
        'profile', 'coworking', 'PRE_LEASED_CURRENT_RENT', 'projectUnitId', 'IS_FRESH',
        'IS_DEFAULT_DEALER_IMAGE', 'REPORTED', 'POSTING_DATE__U', 'shortlisted',
        'possessionStatus', 'locationHighlights', 'PHOTO_URL', 'LISTING', 'GROUP_META',
        'caraousalImages', 'FEATURES', 'PROPERTY_TYPE__U', 'REGISTER_DATE__U', 'OVERLOOKING',
        'FOMO', 'RTOV_BOOKED', 'configSummary', 'OWNTYPE', 'BOOKING_AMOUNT', 'IS_FSL', 'OFFER_TAGS',
        'preference', 'CITY_ID', 'CONTACT_CITY_NAME', 'PRIMARY_TAGS', 'PROP_HEADING', 'HAVEPHOTO',
        'configCardTuples', 'UPDATE_DATE__U', 'FORMATTED_PRICE', 'DEALER_PHOTO_URL', 'BROKERAGE',
        'IS_NEW_LAUNCH', 'PROPERTY_NUMBER', 'GROUP_NAME', 'brochure', 'RESERVED_PARKING',
        'CONTACT_COMPANY_NAME', 'POSTING_DATE', 'PREFERENCE', 'GREEN_TAGS', 'AMENITIES',
        'subHeading', 'VIDEO_FILENAME', 'SPID', 'PAGE', 'tags', 'SOCIETY_NAME', 'MEDIUM_PHOTO_URL',
        'FACING', 'GATED', 'UPDATE_DATE', 'SHORTLISTED', 'IS_POSTER_RERA_REGISTERED',
        'LOCALITY_WO_CITY', 'SHOW_BSP', 'SHOW_WHATSAPP_ICON', 'CONF_ID', 'IS_PRE_LEASED',
        'CLASS', 'MASK_CONTACT', 'landingPage', 'localityLabel', 'builderName', 'AGE',
        'REGISTERED_DAYS', 'REGISTER_DATE', 'EXPIRY_DATE', 'FD', 'RERA_REGISTRATION_ID',
        'FSL_Data', 'PROP_ID', 'VERIFIED', 'ALT_TAG', 'SELF_VERIFIED', 'CLASS_HEADING',
        'VALUE_LABEL', 'SECONDARY_TAGS', 'PROPERTY_IMAGES', 'THUMBNAIL_IMAGES', 'SPRITE_IMAGES',
        'xid', 'heading', 'coverImage', 'DEPOSIT_TYPE', 'PROP_NAME', 'PRICE_SQFT',
        'MIN_PRICE', 'MAX_PRICE', 'MIN_AREA_SQFT', 'MAX_AREA_SQFT', 'MIN_AREA', 'MAX_AREA'
    ],
    inplace=True
)

In [15]:
df.duplicated().sum()

14

In [16]:
df.drop_duplicates(ignore_index=True, inplace=True)

In [17]:
df[['AREA','PRICE']].head(2)

,AREA,PRICE
0,1134-1215 sq.ft.,42 - 45 L
1,2052-2250 sq.ft.,1 - 1.1 Cr


In [18]:
df = df.iloc[2:]
df.reset_index(drop=True, inplace=True)

In [19]:
(df['PRICE']=='Price on Request').sum()

73

In [20]:
df.drop(df[df['PRICE'].str.lower() == 'price on request'].index, inplace=True)
df.reset_index(drop=True, inplace=True)

## Correcting Columns, Introducing New Features, and Necessary Drops

- Now, dataset contains 12,409 entries and 42 different pieces of information. 
- I've corrected the `AREA` column by removing the 'sq.ft.' string from it.
- In the `PRICE` column, I've made corrections by replacing 'Cr' or 'L' with the appropriate numerical values.
- I've introduced a new feature called `AREA_TYPE`, which indicates the type of area mentioned in the `AREA` column.
- To clean up the dataset, I've removed 20 unnecessary columns

In [21]:
df.shape

(12409, 42)

In [22]:
df['AREA'] = df['AREA'].str.replace(' sq.ft.', '')

In [23]:
def convert_price(price):
    if np.nan is price:
        return price
    elif 'Cr' in price:
        price = price.replace('Cr', '')
        return float(price) * 10000000
    elif 'L' in price:
        price = price.replace('L', '')
        return float(price) * 100000

df['PRICE'] = df['PRICE'].apply(convert_price)

In [24]:
temp_df = df.copy()

temp_df['AREA_int'] = temp_df['AREA'].fillna(-1).astype(int)
temp_df['CARPET_SQFT_int'] = temp_df['CARPET_SQFT'].fillna(-1).astype(int)
temp_df['BUILTUP_SQFT_int'] = temp_df['BUILTUP_SQFT'].fillna(-1).astype(int)
temp_df['SUPERBUILTUP_SQFT_int'] = temp_df['SUPERBUILTUP_SQFT'].fillna(-1).astype(int)
temp_df['SUPER_SQFT_int'] = temp_df['SUPER_SQFT'].fillna(-1).astype(int)

area_types = []

for index, row in temp_df.iterrows():
    area = row['AREA_int']
    if area == row['CARPET_SQFT_int']:
        area_types.append('CARPET_SQFT')
    elif area == row['BUILTUP_SQFT_int']:
        area_types.append('BUILTUP_SQFT')
    elif area == row['SUPERBUILTUP_SQFT_int']:
        area_types.append('SUPERBUILTUP_SQFT')
    elif area == row['SUPER_SQFT_int']:
        area_types.append('SUPER_SQFT')
    else:
        area_types.append('Unknown')

df['AREA_TYPE'] = area_types

In [25]:
df.drop(
    columns = [
        'AREA_UNIT', 'COM_AREA_TYPE', 'SECONDARY_AREA', 'SUPER_AREA', 'SUPERAREA_UNIT',
        'SUPER_SQFT', 'SUPERAREA_UNIT__U', 'CARPET_AREA', 'CARPETAREA_UNIT', 'CARPET_SQFT',
        'SUPERBUILTUP_AREA', 'SUPERBUILTUPAREA_UNIT', 'SUPERBUILTUP_SQFT',
        'CARPETAREA_UNIT__U', 'SUPERBUILTUPAREA_UNIT__U', 'BUILTUP_AREA', 'BUILTUP_SQFT',
        'TOTAL_LANDMARK_COUNT', 'FORMATTED_LANDMARK_DETAILS', 'MAP_DETAILS', 'location'
    ],
    inplace=True
)

## Dataset Reduction and Focus on Residential Apartments

- I've reduced the dataset to 12,409 entries and 22 different pieces of information..
- Among the properties, I've identified 8 different types.
- However, for now, I'm only interested in **Residential Apartment**.
- So, I've extracted other specific category into a separate CSV file for future use.
- Currently, I've solely focus on the 'Residential Apartment' category, which consists of 9,291 entries and 22 different pieces of information.

In [26]:
df.shape

(12409, 22)

In [27]:
df['PROPERTY_TYPE'].value_counts()

PROPERTY_TYPE
Residential Apartment        9291
Independent House/Villa      1851
Residential Land              752
Independent/Builder Floor     164
Farm House                     38
Studio Apartment               25
Other                          13
Serviced Apartments             6
Name: count, dtype: int64

In [28]:
residential_apartment_df = df[df['PROPERTY_TYPE']=='Residential Apartment'].copy()

In [29]:
residential_apartment_df.shape

(9291, 22)

## Data Cleaning and Preparation for Residential Apartment Dataset Analysis

- Residential Apartment Property consists of 9,291 entries and 22 different pieces of information.
- I removed 5 rows where any of these columns were null: `BALCONY_NUM`, `BEDROOM_NUM`, `FLOOR_NUM`, `TOTAL_FLOOR`.
- I filled missing values in certain columns:
   - For `BATHROOM_ATTACHED` and `BALCONY_ATTACHED`, if there was no information, I assumed 'Y' (Yes).
   - For `CORNER_PROPERTY`, if there was no information, I assumed 'N' (No).
   - For `TOP_USPS`, if there was no information, I marked it as 'INFO NOT AVAILABLE'.
- I introduced a new feature called `FURNISH_LABEL`, if there was no information, I marked it as 'INFO NOT AVAILABLE'.
- To clean up the dataset, I removed 1 unnecessary column.
- I noticed an error in the `CITY` column, where 'Ahmedabad' was incorrectly labeled. I corrected it to 'Gandhinagar'.
- I created a function called `map_floor_num` to convert the values in the `FLOOR_NUM` column into numeric format:
   - If the value is a number, I converted it to an integer.
   - If the value is 'G', I set it as 0.
   - If the value is 'B' or 'L', I set it as -1.
   - Otherwise, I marked it as null.
- I converted the values in the `AREA` column to float data type.
- I saved the modified dataframe as a CSV file named `Residential_Apartment.csv`.
- This file is now prepared for Exploratory Data Analysis (EDA).
- `Residential_Apartment.csv` consists of 9286 entries and 22 different pieces of information.

In [30]:
residential_apartment_df.shape

(9291, 22)

In [31]:
residential_apartment_df.isnull().sum().loc[lambda x: x!=0].sort_values()

BEDROOM_NUM             1
FLOOR_NUM               1
TOTAL_FLOOR             1
BALCONY_NUM             5
BATHROOM_ATTACHED    2021
BALCONY_ATTACHED     2021
TOP_USPS             3042
CORNER_PROPERTY      5822
dtype: int64

In [32]:
residential_apartment_df['LOCALITY'] = residential_apartment_df['LOCALITY'].str.capitalize()

In [33]:
residential_apartment_df[(residential_apartment_df [['BALCONY_NUM', 'BEDROOM_NUM', 'FLOOR_NUM', 'TOTAL_FLOOR']].isnull()).any(axis=1)].shape[0]

5

In [34]:
residential_apartment_df.dropna(subset=['BALCONY_NUM', 'BEDROOM_NUM', 'FLOOR_NUM', 'TOTAL_FLOOR'], inplace=True, ignore_index=True)

In [35]:
residential_apartment_df['CORNER_PROPERTY'].value_counts()

CORNER_PROPERTY
Y    3468
Name: count, dtype: int64

In [36]:
residential_apartment_df['CORNER_PROPERTY'] = residential_apartment_df['CORNER_PROPERTY'].fillna('N')
residential_apartment_df['TOP_USPS'] = residential_apartment_df['TOP_USPS'].fillna('INFO NOT AVAILABLE')

In [37]:
residential_apartment_df ['FORMATTED'] = residential_apartment_df ['FORMATTED'].apply(ast.literal_eval)

def extract_furnish_label(row):
    if 'FURNISH_LABEL' in row and row['FURNISH_LABEL']:
        return row['FURNISH_LABEL']
    else:
        return 'INFO NOT AVAILABLE'

residential_apartment_df['FURNISH_LABEL'] = residential_apartment_df['FORMATTED'].apply(extract_furnish_label)

In [38]:
residential_apartment_df['FURNISH_LABEL'].value_counts()

FURNISH_LABEL
Unfurnished           4898
Furnished             1872
Semifurnished         1869
INFO NOT AVAILABLE     647
Name: count, dtype: int64

In [39]:
residential_apartment_df['CITY'].value_counts()

CITY
Ahmedabad West     4831
Ahmedabad North    2452
Ahmedabad East     1317
Gandhinagar         608
Ahmedabad South      77
Ahmedabad             1
Name: count, dtype: int64

In [40]:
residential_apartment_df.loc[residential_apartment_df['CITY'] == 'Ahmedabad ', 'CITY'] = 'Gandhinagar'

In [41]:
def map_floor_num(value):
    if value.isdigit():  
        return int(value)
    elif value == 'G':
        return 0
    elif value == 'B' or value == 'L':
        return -1 
    else:
        return None 

residential_apartment_df['FLOOR_NUM'] = residential_apartment_df['FLOOR_NUM'].apply(map_floor_num)

In [42]:
# Define categories based on number of floors
def categorize_floor_count(floor):
    if floor <= 5:
        return 'Low-rise buildings'
    elif floor <= 15:
        return 'Mid-rise buildings'
    elif floor <= 25:
        return 'High-rise buildings'
    else:
        return 'Skyscrapers'

# Apply categorization to the 'TOTAL_FLOOR' column
residential_apartment_df['BUILDING_TYPE'] = residential_apartment_df['TOTAL_FLOOR'].apply(categorize_floor_count)

In [43]:
residential_apartment_df['AREA'] = residential_apartment_df['AREA'].astype(float)

In [44]:
conversion_factors = {
    'BUILTUP_SQFT': 1.1,  # For example, 1 BUILTUP_SQFT is equivalent to 1.1 SUPERBUILTUP_SQFT
    'CARPET_SQFT': 1.2   # For example, 1 CARPET_SQFT is equivalent to 1.2 SUPERBUILTUP_SQFT
}

# Convert 'AREA' values for BUILTUP_SQFT and CARPET_SQFT to SUPERBUILTUP_SQFT
for index, row in residential_apartment_df.iterrows():
    if row['AREA_TYPE'] == 'BUILTUP_SQFT':
        residential_apartment_df.at[index, 'AREA'] *= conversion_factors['BUILTUP_SQFT']
    elif row['AREA_TYPE'] == 'CARPET_SQFT':
        residential_apartment_df.at[index, 'AREA'] *= conversion_factors['CARPET_SQFT']

In [45]:
print(residential_apartment_df[residential_apartment_df['AREA']>3000].shape[0]+residential_apartment_df[residential_apartment_df['AREA']<500].shape[0])
indices_to_drop = residential_apartment_df[(residential_apartment_df['AREA'] > 3000) | (residential_apartment_df['AREA'] < 500)].index
residential_apartment_df.drop(indices_to_drop, inplace=True)
residential_apartment_df.reset_index(drop=True, inplace=True)

2005


In [46]:
print(residential_apartment_df[residential_apartment_df['PRICE']>15000000].shape[0] + residential_apartment_df[residential_apartment_df['PRICE'] < 1000000].shape[0])
indices_to_drop = residential_apartment_df[(residential_apartment_df['PRICE'] > 15000000) | (residential_apartment_df['PRICE'] < 1000000)].index
residential_apartment_df.drop(indices_to_drop, inplace=True)
residential_apartment_df.reset_index(drop=True, inplace=True)

627


In [47]:
print(residential_apartment_df[residential_apartment_df['BEDROOM_NUM']>=5].shape[0])
indices_to_drop = residential_apartment_df[residential_apartment_df['BEDROOM_NUM']>=5].index
residential_apartment_df.drop(indices_to_drop, inplace=True)
residential_apartment_df.reset_index(drop=True, inplace=True)

6


In [48]:
residential_apartment_df.drop(
    columns = [
        'FORMATTED', 'CLASS_LABEL', 'IS_DEALER_RERA_REGISTERED', 'PRODUCT_TYPE',
        'PROPERTY_TYPE', 'PRICE_PER_UNIT_AREA', 'TOTAL_FLOOR', 'BATHROOM_ATTACHED', 'BALCONY_ATTACHED'
    ],
    inplace=True
)

In [49]:
residential_apartment_df.shape

(6648, 15)

In [50]:
column_order = [
    "DESCRIPTION",
    "CITY",
    "LOCALITY",
    "BEDROOM_NUM",
    "BATHROOM_NUM",
    "BALCONY_NUM",
    "FLOOR_NUM",
    "BUILDING_TYPE",
    "AREA",
    "PRICE",
    "TOP_USPS",
    "CORNER_PROPERTY",
    "FURNISH_LABEL",
    "PROP_DETAILS_URL",
]

In [51]:
residential_apartment_df.shape

(6648, 15)

In [52]:
residential_apartment_df = residential_apartment_df[column_order]

In [53]:
residential_apartment_df.isnull().sum().sum()

0

In [54]:
residential_apartment_df.duplicated().sum()

0

In [55]:
residential_apartment_df.to_csv('Residential_Apartment.csv', index=False)